In [ ]:
using JuMP
using CPLEX
using Distributions
using MathProgBase


In [ ]:
srand(1) #para usar una única semilla y repetir siempre los mismos números aleatorios

A=[44.7  1411   2.0   365  0     55.4  33.3  441  0;
    36   897    1.7   99   30.9  17.4  7.9   106  0;
    8.4  422    15.1  9    26    3     23.5  11   60;
    20.6 17     0.6   6    55.8  0.2   0     0    0;
    7.4  448    16.4  19   28.1  0.8   10.3  4    0;
    15.7 661    1     48   0     9.6   8.1   471  0;
    41.7 0      0     0    0.2   0     0.5   5    0;
    2.2  333    0.2   139  169.2 6.4   50.8  316  525;
    4.4  249    0.3   37   0     18.2  3.6   79   0;
    5.8  705    6.8   45   3.5   1     4.9   209  0;
    2.4  138    3.7   80   69    4.3   5.8   37   862;
    2.6  125    4     36   7.2   9     4.5   26   5369;
    5.8  166    3.8   59   16.6  4.7   5.9   21   1184;
    14.3 336    1.8   118  6.7   29.4  7.1   198  2522;
    1.1  106    0     138  918.4 5.7   13.8  33   2755;
    9.6  138    2.7   54   290.7 8.4   5.4   83   1912;
    8.5  87     1.7   173  86.8  1.2   4.3   55   57;
    12.8 99     2.5   154  85.7  3.9   4.3   65   257;
    17.4 1055   3.7   459  5.1   26.9  38.2  93   0;
    26.9 1691   11.4  792  0     38.4  24.6  217  0]

A=A';

b=[30,70,0.8,12,5,1.8,2.7,18,75]

c=ones(20,1);

function randomcoef(c)
    cl=zeros(length(c));
    cu=zeros(length(c));
    for i=1:length(c)
        cl[i]=c[i]-(c[i]*rand()/2);
        cu[i]=c[i]+(c[i]*rand()/2);
    end
    return cl,cu
end
c_min,c_max=randomcoef(c)
println("c_min: ",c_min)
println("c_max: ",c_max)

In [ ]:
function buildinstance(n,m)
    #Vector de costos de primera etapa
    c=zeros(n)
    for i=1:n
        min=round(rand()*10)
        max=min+1+round(rand()*min*10)
        c[i]=round(rand(Uniform(min,max)))
        if c[i]==0
            c[i]=1
        end
    end

    #A matrix
    A=round(rand(1,n)*rand(Uniform(1,30)))
    for i=2:m
        min=round(rand()*rand(Uniform(1,30)))
        max=min+1+round(rand()*min*rand(Uniform(1,30)))
        A=vcat(A,round(rand(Uniform(min,max),1,n)))
    end

    #Set b vector
    incap=sum(A,2)
    b=round((incap.*round(rand(Uniform(5,20),m,1))).*rand(m))
    return c,A,b
end

In [ ]:
c,A,b=buildinstance(10000,1000)
c_min,c_max=randomcoef(c)
b=b/10000

In [ ]:
function solveScenario(A,b,c,solver)
    n=length(c)
    m=size(A,1)
    submodel = Model(solver = solver)
    @variable(submodel, x[1:n]>=0)
    @objective(submodel,Min, sum(c[j]*x[j] for j in 1:n) )
    
    @constraint(submodel, req[i = 1:m], sum(A[i,j]*x[j] for j in 1:n) >= b[i])
    status = solve(submodel)
    #print("solucion: ",status,"\n\n\n")
    valor=getobjectivevalue(submodel)
    return valor
end


In [ ]:
function solveLP(A,b,c,solver)
    n=length(c)
    m=size(A,1)
    submodel = Model(solver = solver)
    @variable(submodel, x[1:n]>=0)
    @objective(submodel,Min, sum(c[j]*x[j] for j in 1:n) )
    
    @constraint(submodel, req[i = 1:m], sum(A[i,j]*x[j] for j in 1:n) >= b[i])
    status = solve(submodel)
    #print("solucion: ",status,"\n\n\n")
    valor=getobjectivevalue(submodel)
    return valor
end

#Subrutina para fijar el límite superior
function solveLP(A,b,c,solver,v,max)
    n=length(c)
    m=size(A,1)
    submodel = Model(solver = solver)
    @variable(submodel, x[1:n]>=0)
    @objective(submodel,Min, sum(c[j]*x[j] for j in 1:n) )
    
    @constraint(submodel, req[i = 1:m], sum(A[i,j]*x[j] for j in 1:n) >= b[i])
    @constraint(submodel, sum(v[j]*x[j] for j in 1:n)==max)
    status = solve(submodel)
    #print("solucion: ",status,"\n\n\n")
    valor,sol=getobjectivevalue(submodel),getvalue(x)
    return valor,sol
end
#preproceso para determinar límites superiores
xmax=zeros(length(c));
for j=1:length(c)
    v=zeros(length(c));
    v[j]=1;
    maxim=b[A[:,j].>0]./A[:,j][A[:,j].>0]
    xm=maximum(maxim)
    #valor,sol=solveLP(A,b,c,CplexSolver(CPX_PARAM_SCRIND=0),v,xm)
    #validación
    #Ar=A[:,setdiff(range(1,size(A,2)),j)]*sol[setdiff(range(1,size(A,2)),j)]
    #pos=find(x->x==xm,b./A[:,j])
    #pos1=find(x->x==xm,maxim)
    #new=xm-Ar[pos]
    #if all(maxim[setdiff(range(1,length(maxim)),pos1)].<new)
    #    xmax[j]=new[1]
    #else
    #    xmax[j]=maximum(maxim[setdiff(range(1,length(maxim)),pos1)])
    #end
    xmax[j]=xm
end
println(xmax)

In [ ]:
#Subproblem (NP-hard) function 
function SubproblemLP(cup,cun,solver,fix1,fix2,n::Int64,xmax,xbar,b)
    r=size(A,1)
    tol=1e-6
    m=Model(solver = solver)
    
    @variable(m,yup[1:n]>=0)
    @variable(m,yun[1:n]>=0)
    @variable(m,zup[1:n],Bin)
    @variable(m,zun[1:n],Bin)

    #function infocallback(cb)
    #    node      = MathProgBase.cbgetexplorednodes(cb)
    #    obj       = MathProgBase.cbgetobj(cb)
    #    bestbound = MathProgBase.cbgetbestbound(cb)
    #    push!(bbdata, NodeData(time(),node,obj,bestbound))
    #end
    #addinfocallback(m, infocallback, when = :Intermediate)
    function infocallback(cb)
        #obj       = MathProgBase.cbgetobj(cb)
        println("\n************************ PIPA\n ")
    end
    addlazycallback(m, infocallback)

    ind=[]
    for j=1:n
        if fix1[j]>=1-tol
            #println("upper")
            setlowerbound(zup[j],1) #fuerza a que valga 1
            setupperbound(zun[j],0) #fuerza a que valga 0
            setupperbound(yup[j],xmax[j]-xbar[j])
            setupperbound(yun[j],0)
        else
            if fix2[j]>=1-tol
                #println("under")
                setupperbound(zup[j],0)
                setlowerbound(zun[j],1)
                setupperbound(yup[j],0)
                setupperbound(yun[j],xbar[j])
            else
                push!(ind,j)
                #println("Algo pasa")
            end
        end
    end
    
    @objective(m,Max,sum(cup[j]*yun[j] for j in 1:n) - sum(cun[j]*yup[j] for j in 1:n))
    
    @constraint(m, req[i = 1:r], sum(A[i,j]*xbar[j] for j in 1:n) + sum(A[i,j]*yup[j] for j in 1:n) - sum(A[i,j]*yun[j] for j in 1:n) >= b[i])
    @constraint(m, plusz[j in ind], zup[j]+zun[j]==1)
    @constraint(m, plusy[j in ind], (xmax[j]-xbar[j])*zup[j] >= yup[j])
    @constraint(m, minusy[j in ind], xbar[j]*zun[j] >= yun[j])
    
    #print("-----------------SUB-------------------")
    #println(m)
    #print("-----------------SUB-------------------")
    
    status = solve(m)
    print("solucion: ",status,"\t",getobjectivevalue(m),"\n\n\n")
    valor=getobjectivevalue(m)
    zu=getvalue(zup)
    #println("valor z ",zu)
    cost_scen=zeros(Float64,n)
    for i=1:n
        if zu[i]>=1-tol
            cost_scen[i]=cun[i]
        else
            cost_scen[i]=cup[i]
        end
    end    
    return valor,cost_scen
end

In [ ]:
function solveRLP(A,b,cminus, cplus, xmax, n::Int64, solver=CplexSolver(CPX_PARAM_LPMETHOD=1,CPX_PARAM_SCRIND=0))
    #We initilize the algorithm solving the original LP using the midpoint scenario
    r=size(A,1)
    cMidPoint=zeros(Float64,n)
    for i in 1:n
        cMidPoint[i]=(cminus[i]+cplus[i])/2
    end
    optimalCost = solveScenario(A,b,cMidPoint,CplexSolver(CPX_PARAM_SCRIND=0))
    #rhs = - optimalCost  [JPG]
    
    #We set the MinMax Regret problem for the first iteration
    m = Model(solver = solver)
    @variable(m, x[1:n]>=0)
    @variable(m,θ)

    @objective(m, Min, θ )
    
    @constraint(m, req[i = 1:r], sum(A[i,j]*x[j] for j in 1:n) >= b[i])
    
    #@constraint(m,θ - sum(cMidPoint[j]*x[j] for j in 1:n) >=rhs)   [JPG]
    @constraint(m,θ >= sum(cMidPoint[j]*x[j] for j in 1:n) - optimalCost  )
    #print(m)
    

    status = solve(m)
    #println(status)
    #println(getvalue(x))
    println("LB: ", getobjectivevalue(m))
    
    #Max admisible difference between consecutive solutions of our MinMax Regret Problem
    tol=0.0001
    
    #Initial bounds
    LB=getobjectivevalue(m)
    if LB<=tol
        LBprime=-1
    else
        LBprime=-10*LB
    end
    
    #Tolerance to 0.0
    EPSILON=0.0001
    it=1
    
    conteo=0
    
    println("Prueba LB ",abs(LB-LBprime))
    while abs(LB-LBprime)>=tol #conteo<=10 # [JPG]
        #Generating a new cut
        rhs=0
        x_val = getvalue(x)
        #Analizing x we can fix some value of z+ and z- in the combinatorial subproblem
        fixup=zeros(Int64,n)
        fixun=zeros(Int64,n)
        for i in 1:n
            if x_val[i]<=EPSILON 
                fixun[i]=1
            else
                if x_val[i]>=(xmax[i]-EPSILON)
                    fixup[i]=1
                end
            end
        end
        #println("\t\tfixup: ",fixup,"\n")
        #println("\t\tfixun: ",fixun,"\n")
        tt=time()
#        optimalCost,cost = SubproblemLP(cplus,cminus,CplexSolver(CPX_PARAM_SCRIND=0),fixun,fixup,n,xmax,x_val,b)
        optimalCost,cost = SubproblemLP(cplus,cminus,CplexSolver(),fixun,fixup,n,xmax,x_val,b)
        #println("Tiempo subproblema: ", time()-tt)
        print("Costo optimo subproblema: ", optimalCost)
        println(" vector cost: ",cost)
        #New cut
        tt1=time()
        rhs=solveScenario(A,b,cost,CplexSolver(CPX_PARAM_SCRIND=0))
        #println("Tiempo RHS: ", time()-tt1)
        #@constraint(m, θ - sum(cost[j]*x[j] for j in 1:n) >= -rhs ) [JPG]
        @constraint(m,θ >= sum(cost[j]*x[j] for j in 1:n) - rhs  )
        
        #print("-----------------######-------------------")
        #print(m)
        #print("-----------------######-------------------")
        
        tt2=time()
        status = solve(m)
        #println("Tiempo LP: ", time()-tt2)
        println(status)
        println("θ ",getvalue(θ),"\tvariables master: ",getvalue(x))
        #Updating bounds
        LBprime=LB
        LB=getobjectivevalue(m)
        
        #Counting consecutive LPs with small improvements (tol) in our bounds
        #if abs(LB-LBprime)<=tol
        #    conteo += 1
        #else
        #    conteo = 1
        #end
        #println("LB: ", getobjectivevalue(m))
        it += 1
        println("GAP entre soluciones consecutivas ", abs(LB-LBprime))
    end
    
    println("TotalIteraciones: ", it)
    println("maxRegret: ",getobjectivevalue(m))
    println("Solución",getvalue(x))
end

In [ ]:
solveRLP(A,b,c_min, c_max, xmax, length(c))

In [ ]:
type NodeData
    time::Float64  # in seconds since the epoch
    obj::Float64
end

#Subproblem (NP-hard) function 
function SubproblemLPim(cup,cun,solver,fix1,fix2,n::Int64,xmax,xbar,b,zopt,tt,scens,timemax,mindis)
    r=size(A,1)
    tol=1e-6
    m=Model(solver = solver)
    
    @variable(m,yup[1:n]>=0)
    @variable(m,yun[1:n]>=0)
    @variable(m,zup[1:n],Bin)
    @variable(m,zun[1:n],Bin)
    
    function infocallback(cb)
        #obj       = MathProgBase.cbgetobj(cb)
        println("\n************************ PIPA\n ")
    end
    addlazycallback(m, infocallback)
    
    ind=[]
    for j=1:n
        if fix1[j]>=1-tol
            setvalue(zup[j],1)
            setvalue(zun[j],0)
            setupperbound(yup[j],xmax[j]-xbar[j])
            setupperbound(yun[j],0)
        else
            if fix2[j]>=1-tol
                #println("under")
                setvalue(zup[j],0)
                setvalue(zun[j],1)
                setupperbound(yup[j],0)
                setupperbound(yun[j],xbar[j])
            else
                push!(ind,j)
                #println("Algo pasa")
            end
        end
    end
    
    @objective(m,Max,sum(cup[j]*yun[j] for j in 1:n) - sum(cun[j]*yup[j] for j in 1:n))
    
    @constraint(m, req[i = 1:r], sum(A[i,j]*xbar[j] for j in 1:n) + sum(A[i,j]*yup[j] for j in 1:n) - sum(A[i,j]*yun[j] for j in 1:n) >= b[i])
    @constraint(m, plusz[j in ind], zup[j]+zun[j]==1)
    @constraint(m, plusy[j in ind], (xmax[j]-xbar[j])*zup[j] >= yup[j])
    @constraint(m, minusy[j in ind], xbar[j]*zun[j] >= yun[j])
    
    #print("-----------------SUB-------------------")
    #println(m)
    #print("-----------------SUB-------------------")
    
    #function getcursol(cb)
    #    zval=getvalue(zup)
    #    obj=MathProgBase.cbgetobj(cb)
    #    cost_scen=zeros(length(zval))
    #    println("Solución parcial: ", obj)
    #    println("zval ", zval)
    #    
    #    intsol, cost_scen = check_solution(zval,cun,cup)
    #    println("Es entera la solución:", intsol)
    #    
    #    setdis = check_distance(scens,cost_scen,mindis)
    #    println("Cumple condiciones de distancia: ", setdis)
    #    
    #    if (obj-zopt)/zopt>0.1 && intsol==true && setdis==true
    #        println("Solucion satisface condiciones para agregar el escenario")
    #        return JuMP.StopTheSolver
    #    end
    #end
    
    #if time()-tt<=timemax
    #    addlazycallback(m,getcursol)
    #end
    #bbdata = NodeData[]
    
    
    #function infocallback(cb)
    #    obj       = MathProgBase.cbgetobj(cb)
    #    push!(bbdata, NodeData(time(),obj))
    #end
    #addinfocallback(m, infocallback, when = :MIPNode)
    
    #println(bbdata)

    status = solve(m)
    #print("solucion: ",status,"\n\n\n")
    valor=getobjectivevalue(m)
    zu=getvalue(zup)
    #println("valor z ",zu)
    cost_scen=zeros(Float64,n)
    for i=1:n
        if zu[i]>=1-tol
            cost_scen[i]=cun[i]
        else
            cost_scen[i]=cup[i]
        end
    end     
    return valor,cost_scen
end

In [ ]:
function solveRLPim(A,b,cminus, cplus, xmax, n::Int64,timemax,mindis,solver=CplexSolver(CPX_PARAM_LPMETHOD=1))
    #We initilize the algorithm solving the original LP using the midpoint scenario
    r=size(A,1)
    cMidPoint=zeros(Float64,n)
    for i in 1:n
        cMidPoint[i]=(cminus[i]+cplus[i])/2
    end
    optimalCost = solveLP(A,b,cMidPoint,CplexSolver(CPX_PARAM_SCRIND=0))
    rhs = - optimalCost 
    
    #We set the MinMax Regret problem for the first iteration
    m = Model(solver = solver)
    @variable(m, x[1:n]>=0)
    @variable(m,θ)

    @objective(m, Min, θ )
    
    @constraint(m, req[i = 1:r], sum(A[i,j]*x[j] for j in 1:n) >= b[i])
    
    @constraint(m,θ - sum(cMidPoint[j]*x[j] for j in 1:n) >=rhs)        
    #print(m)
    
    status = solve(m)
    #println(status)
    #println(getvalue(x))
    println("LB: ", getobjectivevalue(m))
    
    #Max admisible difference between consecutive solutions of our MinMax Regret Problem
    tol=0.0001
    
    #Initial bounds
    LB=getobjectivevalue(m)
    if LB<=tol
        LBprime=-1
    else
        LBprime=-10*LB
    end
    
    #Tolerance to 0.0
    EPSILON=0.0001
    it=1
    
    conteo=0
    
    println("Prueba LB",abs(LB-LBprime))
    while abs(LB-LBprime)>=tol#conteo<=10
        #Generating a new cut
        rhs=0
        x_val = getvalue(x)
        #Analizing x we can fix some value of z+ and z- in the combinatorial subproblem
        fixup=zeros(Int64,n)
        fixun=zeros(Int64,n)
        for i in 1:n
            if x_val[i]<=EPSILON 
                fixun[i]=1
            else
                if x_val[i]>=(xmax[i]-EPSILON)
                    fixup[i]=1
                end
            end
        end
        #println("\t\t",fixup,"\n")
        #println("\t\t",fixun,"\n")
        if it==1
            tta=time()
            optimalCost,cost = SubproblemLP(cplus,cminus,CplexSolver(),fixun,fixup,n,xmax,x_val,b)
            println("Tiempo subproblema: ", time()-tta)
            scens=cost
        else
            tta=time()
            optimalcost,cost = SubproblemLPim(cplus,cminus,CplexSolver(),fixun,fixup,n,xmax,x_val,b,getobjectivevalue(m),time(),scens,timemax,mindis)
            println("Tiempo subproblema: ", time()-tta)
            scens=vcat(scens,cost)
        end
        println("Costo optimo subproblema: ", optimalCost)
        #New cut
        tt1=time()
        rhs=solveLP(A,b,cost,CplexSolver(CPX_PARAM_SCRIND=0))
        println("Tiempo RHS: ", time()-tt1)
        @constraint(m, θ - sum(cost[j]*x[j] for j in 1:n) >= -rhs )
        
        #print("-----------------######-------------------")
        #print(m)
        #print("-----------------######-------------------")
        
        tt2=time()
        status = solve(m)
        println("Tiempo LP: ", time()-tt2)
        println(status)
        #println(getvalue(x))
        #Updating bounds
        LBprime=LB
        LB=getobjectivevalue(m)
        
        #Counting consecutive LPs with small improvements (tol) in our bounds
        #if abs(LB-LBprime)<=tol
        #    conteo += 1
        #else
        #    conteo = 1
        #end
        println("LB: ", getobjectivevalue(m))
        it += 1
        #print("GAP entre soluciones consecutivas", abs(LB-LBprime))
    end
    
    println("TotalIteraciones: ", it)
    println("maxRegret: ",getobjectivevalue(m))
    println("Solución",getvalue(x))
end

In [ ]:
solveRLPim(A,b,c_min, c_max, xmax, length(c),10,0.1)